# Uniview module for [M82](https://en.wikipedia.org/wiki/Messier_82) 

I will use the  [FIRE](https://galaxies.northwestern.edu/fire-simulations/) isolated galaxy model that we've used for testing [Firefly](https://github.com/ageller/Firefly) here.  That simulation’s “proper name” is control_G4_020 and was run by Paul Torrey as part of his nuclear star formation paper.  Note, the bi-conical outflow in the simulation  is due to AGN feedback, so scientifically this isn’t accurate.  But the look should be fine for our purposes. 

*Aaron Geller, 2019*

### Imports and function definitions

In [1]:
#This directory contains all the data needed for the module. It should be in the same directory as the notebook
dataFolder = "data"

import sys, os, shutil, errno, string, urllib

sys.path.append(( os.path.abspath( os.path.join(os.path.realpath("__file__"),  os.pardir, os.pardir) )))
#print(sys.path)
import uvmodlib.v1 as uvmod

### USES Conf Templates

In [2]:
GalaxyTemplate = """mesh
{  
    data gas    ./modules/$folderName/FIREgas.raw
    data stars    ./modules/$folderName/FIREstars.raw
    data  quad ./modules/$folderName/quad.3ds

    dataHints gas disablePostprocessing
    dataHints stars disablePostprocessing

    data center    ./modules/$folderName/center.raw

    cullRadius $cr
    glslVersion 330
    enableTypeCheck true
    
    propertyCollection 
    {        
        __objectName__
        {           

            
            vec1f gasAlpha 0.1 | public | desc "gas alpha" | widget slider | range 0 1
            vec1f gasPsize 0.2 | public | desc "gas particle size" | widget slider | range 0 10
            vec1f gasHighVPsize 2 | public | desc "gas particle size  for v > vMax" | widget slider | range 0 10
            vec1f gasHighVAlpha 0.07 | public | desc "gas particle alpha for v > vMax" | widget slider | range 0 10
            vec1f gasVMin 0. | public | desc "gas velocity colormap min" | widget slider | range 0 500
            vec1f gasVMax 500. | public | desc "gas velocity colormap max" | widget slider | range 500 1000
            
            vec1f starsAlpha 0.015 | public | desc "stars alpha" | widget slider | range 0 1
            vec1f starsPsize 0.80 | public | desc "stars particle size" | widget slider | range 0 10
            vec1f starsHighVPsize 0.80 | public | desc "stars particle size for v > vMax" | widget slider | range 0 10
            vec1f starsHighVAlpha 0.015 | public | desc "stars particle alpha for v > vMax" | widget slider | range 0 10
            vec1f starsVMin 0. | public | desc "stars velocity colormap min" | widget slider | range 0 500
            vec1f starsVMax 400. | public | desc "stars velocity colormap max" | widget slider | range 500 1000
            
            vec1f rotationX 1.2 | public | desc "X rotation angle" | widget slider | range 0 6.283
            vec1f rotationY 3.14 | public | desc "Y rotation angle" | widget slider | range 0 6.283
            vec1f rotationZ 0.0 | public | desc "Z rotation angle" | widget slider | range 0 6.283
            
            vec1f SNRadScale 200 | public | desc "max SN radius" | widget slider | range 0 500
            vec1f SNAlpha 1 | public | desc "SN alpha " | widget slider | range 0 1
            vec1f SNTeff 16000 | public | desc "Teff of for color" | widget slider | range 0 19000
            vec1f SNdlim 0. | public | desc "size of central solid bit" | widget slider | range 0 1
            vec1f SNdpow 0.9 | public | desc "exponential factor in falloff" | widget slider | range 0 10
            vec1f SNdfac 25 | public | desc "multiplicative factor in falloff" | widget slider | range 0 100
            vec3f SNoffset 4.7 2.5 -2.4 | public |desc "offset of the SN from the galaxy center"
            
            vec1f eventTime 0 | public | desc "event time " #| widget slider | range 0 30
            vec1f transitionLength 5 | public | desc "transition length in seconds" 
            bool jump true | public  | desc "jump to time without transition"          
        }
    }


   
    ############# gas particles
    pass
    {
        useDataObject gas
        shader
        {
            type defaultMeshShader
            {
                geometryShader   ./modules/$folderName/galaxy.gs
                vertexShader ./modules/$folderName/galaxy.vs
                fragmentShader   ./modules/$folderName/galaxy.fs
                texture cmap ./modules/$folderName/cmapReds.png
                {             
                    wrapModeS GL_CLAMP_TO_EDGE
                    wrapModeR GL_CLAMP_TO_EDGE
                    colorspace linear
                } 
                stateManagerVar __objectName__.gasPsize  userPsize
                stateManagerVar __objectName__.gasAlpha  userAlpha
                stateManagerVar __objectName__.rotationX  userRotationX
                stateManagerVar __objectName__.rotationY  userRotationY
                stateManagerVar __objectName__.rotationZ  userRotationZ
                
                stateManagerVar __objectName__.gasVMin  vMin
                stateManagerVar __objectName__.gasVMax  vMax
                stateManagerVar __objectName__.gasHighVPsize  highVPsize
                stateManagerVar __objectName__.gasHighVAlpha  highVAlpha
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE false
                    UV_WRITE_MASK_DEPTH true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }  
    }
    
    ############# stars particles
    pass
    {
        useDataObject stars
        shader
        {
            type defaultMeshShader
            {
                geometryShader   ./modules/$folderName/galaxy.gs
                vertexShader ./modules/$folderName/galaxy.vs
                fragmentShader   ./modules/$folderName/galaxy.fs
                texture cmap ./modules/$folderName/cmapBlues.png
                {             
                    wrapModeS GL_CLAMP_TO_EDGE
                    wrapModeR GL_CLAMP_TO_EDGE
                    colorspace linear
                } 
                stateManagerVar __objectName__.starsPsize  userPsize
                stateManagerVar __objectName__.starsAlpha  userAlpha
                stateManagerVar __objectName__.rotationX  userRotationX
                stateManagerVar __objectName__.rotationY  userRotationY
                stateManagerVar __objectName__.rotationZ  userRotationZ

                stateManagerVar __objectName__.starsVMin  vMin
                stateManagerVar __objectName__.starsVMax  vMax
                stateManagerVar __objectName__.starsHighVPsize  highVPsize
                stateManagerVar __objectName__.starsHighVAlpha  highVAlpha

                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE false
                    UV_WRITE_MASK_DEPTH true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }  
    }
    
    
    ############# to hold the time information for SN
    renderTexture
    {
        name                      stateTexture
        width                     1
        height                    1
        numTextures               1
        isPingPong                true
        isPersistent              true
        isFramePersistent         true
        internalTextureFormat     GL_RGB32F
        magnify                   GL_NEAREST
        minify                    GL_NEAREST
    }
    ############# set Transition State
    pass 
    {
        useDataObject quad
        renderTarget
        {
            name stateTexture
            enableColorClear false
        }
         shader
        {
            type defaultMeshShader
            {
                vertexShader   ./modules/$folderName/pass0.vs
                fragmentShader ./modules/$folderName/state.fs
                textureFBO stateTexture stateTexture
                stateManagerVar __objectName__.transitionLength transitionLength 
                stateManagerVar __objectName__.jump jump 
                stateManagerVar __objectName__.eventTime eventTime 
                parameter2f timeRange 0 400
           }
        }      
    }
    
    

    ############# SN (like stellar corona)   
    pass
    {
        useDataObject center
        shader
        {
            type defaultMeshShader
            {
                geometryShader   ./modules/$folderName/SNcorona.gs
                vertexShader ./modules/$folderName/SNcorona.vs
                fragmentShader   ./modules/$folderName/SNcorona.fs
                textureFBO stateTexture stateTexture
                texture bb ./modules/$folderName/bb.png
                {             
                    wrapModeS GL_CLAMP_TO_EDGE
                    wrapModeR GL_CLAMP_TO_EDGE
                    colorspace linear
                }
                
                stateManagerVar __objectName__.SNRadScale  radScale
                stateManagerVar __objectName__.SNTeff  Teff
                stateManagerVar __objectName__.SNAlpha  userAlpha
                stateManagerVar __objectName__.SNdlim  dlim
                stateManagerVar __objectName__.SNdpow  dpow
                stateManagerVar __objectName__.SNdfac  dfac
                stateManagerVar __objectName__.SNoffset  offset
                
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE false
                    UV_WRITE_MASK_DEPTH true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }    
    }

    
}"""


### Galaxy and SN Classes

In [3]:
class GalaxyClass():
    def __init__(self, object):
        self.object = object
        uvmod.Utility.ensurerelativepathexsists("FIREgas.raw",dataFolder)        
        uvmod.Utility.ensurerelativepathexsists("FIREstars.raw",dataFolder)        
        uvmod.Utility.ensurerelativepathexsists("galaxy.gs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("galaxy.fs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("galaxy.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("SNcorona.gs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("SNcorona.fs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("SNcorona.vs",dataFolder)
        self.cr = 1000
    def generatemod(self):
        self.object.setgeometry(self.object.name+"Mesh.usesconf")
        return self.object.generatemod()
    def generatefiles(self, absOutDir, relOutDir):
        fileName = self.object.name+"Mesh.usesconf"
        s = string.Template(GalaxyTemplate)
        f = open(absOutDir+"\\"+fileName, 'w')
        if f:
            f.write(s.substitute(folderName = relOutDir, cr = self.cr))
            f.close()
        uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, absOutDir)

### Object Instantiation

In [4]:
model = GalaxyClass(uvmod.OrbitalObject())
scene = uvmod.Scene()
parentScene = uvmod.Scene()
modinfo = uvmod.ModuleInformation()
generator = uvmod.Generator()

### Specify Settings and generate the module

In [5]:
scene.setname("M82FIRE")
scene.setparent("Extragalactic")
scene.setunit(5e18)
scene.setentrydist(1e5)
scene.setstaticposition(-2.07816693, 1.6584028, 2.27616396)

model.object.setcameraradius(1)
model.object.setcoord(scene.name)
model.object.setname("M82FIRE")
model.object.setguiname("/KavliLecture/Zooniverse/M82FIRE")
model.object.settargetradius(100)
model.object.showatstartup(False)
model.cr = 10

modinfo.setname("M82FIRE")
modinfo.setauthor("Aaron Geller<sup>1</sup>, Laura Trouille<sup>1</sup>, Chris Lintott<sup>2</sup> and the FIRE collaboration<sip>3</sup><br />(1)Adler Planetarium,<br />(2)Oxford University,<br />(3)https://fire.northwestern.edu/")
modinfo.cleardependencies()
modinfo.setdesc("Uniview module for a M82 using FIRE data.  This also contains a SN2014J flash, controlled by eventTime.  The size and alpha follow the AAVSO light curve.")
modinfo.setthumbnail("data/M82ScreenshotThumb.png")
modinfo.setversion("1.0")

generator.generate("M82FIRE",[scene],[model],modinfo)
uvmod.Utility.senduvcommand(model.object.name+".reload")


## To reload

In [6]:
from config import Settings

In [7]:
uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, Settings.uvcustommodulelocation+'\\'+model.object.name)
uvmod.Utility.senduvcommand(model.object.name+".reload;")

Unable to connect to Uniview


### Create colormap texture

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

In [9]:
gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))

def plot_cmap(colormap, fname):
    fig=plt.imshow(gradient, aspect=1, cmap=colormap)
    plt.axis('off')
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    plt.savefig(fname, bbox_inches='tight',pad_inches=0)


plot_cmap('Blues', 'data/cmapBlues.png')
plot_cmap('Reds', 'data/cmapReds.png')


## M82 position 

*From https://en.wikipedia.org/wiki/Messier_82*

In [10]:
from astropy.coordinates import SkyCoord, Distance
from astropy import units

In [11]:
RA = "9h 55m 52.2s" #right ascension
Dec= "69d 40m 47s" #declination
dist = (3500000 *units.pc).to(units.Mpc) #distance

coord = SkyCoord(RA, Dec, dist)

#print(coord.cartesian)
print(coord.galactic.cartesian)


(-2.07816693, 1.6584028, 2.27616396) Mpc


## Position  of SN 2014J in M82

https://en.wikipedia.org/wiki/SN_2014J

In [12]:
RA = "9h 55m 42.217s" #right ascension
Dec= "69d 40m 26.56s" #declination
dist = (3500000 *units.pc).to(units.Mpc) #distance

SNcoord = SkyCoord(RA, Dec, dist)

print(SNcoord.galactic.cartesian)

(-2.07892522, 1.65798914, 2.27577283) Mpc


In [13]:
offset = SNcoord.galactic.cartesian - coord.galactic.cartesian
print(offset)

(-0.00075829, -0.00041366, -0.00039114) Mpc
